In [21]:
from langchain_groq import ChatGroq

In [22]:
llm = ChatGroq(
    model="llama3-70b-8192", 
    temperature=0,
    groq_api_key='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'  
)

response = llm.invoke("The first person to land on the moon was...")
print(response.content)

...Neil Armstrong! On July 20, 1969, Neil Armstrong became the first person to set foot on the moon, famously declaring "That's one small step for man, one giant leap for mankind" as he stepped off the lunar module Eagle onto the moon's surface.


In [23]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://careers.nike.com/retail-district-manager-nike-partner-stores-delhi-india/job/R-60922?utm_campaign=google_jobs_apply&utm_source=google_jobs_apply&utm_medium=organic")
page_data = loader.load().pop().page_content
print(page_data)






















Retail District Manager - Nike Partner Stores (Delhi, India)










































Skip to main content
Open Virtual Assistant










Home


Career Areas


Total Rewards


Life@Nike


Purpose










Language





Select a Language

  Deutsch  
  English  
  Español (España)  
  Español (América Latina)  
  Français  
  Italiano  
  Nederlands  
  Polski  
  Tiếng Việt  
  Türkçe  
  简体中文  
  繁體中文  
  עִברִית  
  한국어  
  日本語  








Careers


















Close Menu







Careers






Chat






                                Home
                            



                                Career Areas
                            



                                Total Rewards
                            



                                Life@Nike
                            



                                Purpose
                            










Jordan Careers







Converse Careers










Language













In [24]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
     """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)

chain_extract = prompt_extract | llm 
res = chain_extract.invoke(input={'page_data':page_data})
type(res.content)

str

In [25]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

[{'role': 'Retail District Manager - Nike Partner Stores',
  'experience': '10+ years',
  'skills': ['Retail Acumen',
   'Relationship Management Skills',
   'High-Profile Communication and Influencing Skills',
   'Team Management Skills'],
  'description': 'Lead, drive, coach, and inspire our store teams. Manage our portfolio of Partner, mono-brand stores in India, and ensure that Strategy is executed throughout the Fleet, while developing territory-specific strategy and targets that ensure premium consumer experiences and joint profitability.'}]

In [26]:
type(json_res)

list

In [27]:
import pandas as pd

df = pd.read_csv("my_portfolio.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [28]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

In [34]:
links = collection.query(query_texts=job['skills'], n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/magento-portfolio'},
  {'links': 'https://example.com/ios-ar-portfolio'}],
 [{'links': 'https://example.com/vue-portfolio'},
  {'links': 'https://example.com/react-portfolio'}],
 [{'links': 'https://example.com/ios-portfolio'},
  {'links': 'https://example.com/ml-python-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/devops-portfolio'}]]

In [35]:
for job in json_res:
    print(job['skills'])


['Retail Acumen', 'Relationship Management Skills', 'High-Profile Communication and Influencing Skills', 'Team Management Skills']


In [36]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Mohan, a business development executive at AtliQ. AtliQ is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of AtliQ 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase Atliq's portfolio: {link_list}
        Remember you are Mohan, BDE at AtliQ. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Enhance Retail Operations with AtliQ's Expertise

Dear [Hiring Manager's Name],

I came across the job description for a Retail District Manager - Nike Partner Stores, and I was impressed by the scope of responsibilities and the skills required to excel in this role. As a business development executive at AtliQ, I believe our company can significantly contribute to the success of your retail operations.

AtliQ specializes in automating business processes, and our expertise can help you streamline store operations, enhance customer experiences, and increase profitability. Our tailored solutions have empowered numerous enterprises to achieve scalability, process optimization, cost reduction, and heightened overall efficiency.

In the retail sector, we have successfully implemented solutions for various clients, including [Magento Portfolio](https://example.com/magento-portfolio) and [iOS Portfolio](https://example.com/ios-portfolio). These projects demonstrate our capabilities i